# 1. import dataset

In [1]:
import numpy as np
import pandas as pd

# 2. import dataset
* input dataset has 110510 rows and 15 columns 
* financial performance is analyzed
* grade is analyzed

In [2]:
df = pd.read_csv('../inputs/fossilfund_dataset.csv')
df = df[['Fund profile: Fund name','Fund profile: Category group','Fund profile: Sustainability mandate','Fund profile: US-SIF member',
         'Fund profile: Fund net assets','Financial performance: Month end trailing returns, year 1','Financial performance: Month end trailing returns, year 3',
         'Financial performance: Month end trailing returns, year 5','Fossil Free Funds: Fossil fuel grade','Deforestation Free Funds: Deforestation grade',
         'Gender Equality Funds: Gender equality grade','Gun Free Funds: Civilian firearm grade','Weapon Free Funds: Military weapon grade',
         'Tobacco Free Funds: Tobacco grade','Prison Free Funds: Prison industrial complex grade']]
df.shape

C:\Users\Lenovo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(110510, 15)

In [3]:
df['Fossil Free Funds: Fossil fuel grade'] = np.where(df['Fossil Free Funds: Fossil fuel grade'].isin(['A','B']),0,
                                                      np.where(df['Fossil Free Funds: Fossil fuel grade'].isin(['F']),2,1))
df['Deforestation Free Funds: Deforestation grade'] = np.where(df['Deforestation Free Funds: Deforestation grade'].isin(['A','B']),0,
                                                               np.where(df['Deforestation Free Funds: Deforestation grade'].isin(['F']),2,1))
df['Gender Equality Funds: Gender equality grade'] = np.where(df['Gender Equality Funds: Gender equality grade'].isin(['A','B']),0,
                                                              np.where(df['Gender Equality Funds: Gender equality grade'].isin(['F']),2,1))
df['Gun Free Funds: Civilian firearm grade'] = np.where(df['Gun Free Funds: Civilian firearm grade'].isin(['A','B']),0,
                                                        np.where(df['Gun Free Funds: Civilian firearm grade'].isin(['F']),2,1))
df['Weapon Free Funds: Military weapon grade'] = np.where(df['Weapon Free Funds: Military weapon grade'].isin(['A','B']),0,
                                                          np.where(df['Weapon Free Funds: Military weapon grade'].isin(['F']),2,1))
df['Tobacco Free Funds: Tobacco grade'] = np.where(df['Tobacco Free Funds: Tobacco grade'].isin(['A','B']),0,
                                                   np.where(df['Tobacco Free Funds: Tobacco grade'].isin(['F']),2,1))
df['Prison Free Funds: Prison industrial complex grade'] = np.where(df['Prison Free Funds: Prison industrial complex grade'].isin(['A','B']),0,
                                                                    np.where(df['Prison Free Funds: Prison industrial complex grade'].isin(['F']),2,1))
df.shape

(110510, 15)

In [4]:
df = df.groupby(['Fund profile: Fund name','Fund profile: Category group','Fund profile: Sustainability mandate',
                 'Fund profile: US-SIF member']).agg({'Fund profile: Fund net assets':'median',
                                                      'Financial performance: Month end trailing returns, year 1':'median',
                                                      'Financial performance: Month end trailing returns, year 3':'median',
                                                      'Financial performance: Month end trailing returns, year 5':'median',
                                                      'Fossil Free Funds: Fossil fuel grade':'median',
                                                      'Deforestation Free Funds: Deforestation grade':'median',
                                                      'Gender Equality Funds: Gender equality grade':'median',
                                                      'Gun Free Funds: Civilian firearm grade':'median',
                                                      'Weapon Free Funds: Military weapon grade':'median',
                                                      'Tobacco Free Funds: Tobacco grade':'median',
                                                      'Prison Free Funds: Prison industrial complex grade':'median'}).reset_index()

df.to_csv('../inputs/1_drop_duplicates.csv', index=False)
df.shape

(75, 15)

# 3. Financial performance
* missing value imputation - with average 
* bins - top 20%ile, mid 20%ile to 50%ile and bottom 50%ile

In [5]:
df1 = df[['Financial performance: Month end trailing returns, year 1','Financial performance: Month end trailing returns, year 3',
          'Financial performance: Month end trailing returns, year 5']]
df1.describe(percentiles=[0.5,0.8]).transpose()

,count,mean,std,min,50%,80%,max
"Financial performance: Month end trailing returns, year 1",75.0,8.915470,11.418853,-17.004200,9.80015,18.113940,37.484015
"Financial performance: Month end trailing returns, year 3",73.0,7.909641,5.450656,-2.488995,8.15726,12.091252,20.513385
"Financial performance: Month end trailing returns, year 5",61.0,9.067412,4.713735,0.344705,9.43515,12.786220,19.721190


In [6]:
df['Financial performance year 1'] = np.where(df['Financial performance: Month end trailing returns, year 1']>18.113940,'a. top',
                                              np.where(df['Financial performance: Month end trailing returns, year 1']<=9.80015,'c. bottom','b. mid'))
df['Financial performance year 3'] = np.where(df['Financial performance: Month end trailing returns, year 3']>12.091252,'a. top',
                                              np.where(df['Financial performance: Month end trailing returns, year 3']<=8.15726,'c. bottom','b. mid'))
df['Financial performance year 5'] = np.where(df['Financial performance: Month end trailing returns, year 5']>12.786220,'a. top',
                                              np.where(df['Financial performance: Month end trailing returns, year 5']<=9.43515,'c. bottom','b. mid'))

df['Financial performance'] = np.where((df['Financial performance year 1']=='a. top') & (df['Financial performance year 3']=='a. top') & (df['Financial performance year 5']=='a. top'), 'a. top',
                                       np.where((df['Financial performance year 1']=='c. bottom') | (df['Financial performance year 3']=='c. bottom') | (df['Financial performance year 5']=='c. bottom'), 'c. bottom', 'b. mid'))

df1 = df.groupby('Financial performance').agg({'Fund profile: Fund name':'count'}).reset_index()
df1

,Financial performance,Fund profile: Fund name
0,a. top,6
1,b. mid,24
2,c. bottom,45


# 4. Grade
* bins < 0.5 then top
* bins >= 0.5 then bottom

In [7]:
df['Fossil fuel grade'] = np.where(df['Fossil Free Funds: Fossil fuel grade']==0,'a. top',
                                   np.where(df['Fossil Free Funds: Fossil fuel grade']==1,'b. mid','c. bottom'))
df['Deforestation grade'] = np.where(df['Deforestation Free Funds: Deforestation grade']==0,'a. top',
                                     np.where(df['Deforestation Free Funds: Deforestation grade']==1,'b. mid','c. bottom'))
df['Gender equality grade'] = np.where(df['Gender Equality Funds: Gender equality grade']==0,'a. top',
                                       np.where(df['Gender Equality Funds: Gender equality grade']==1,'b. mid','c. bottom'))
df['Civilian firearm grade'] = np.where(df['Gun Free Funds: Civilian firearm grade']==0,'a. top',
                                        np.where(df['Gun Free Funds: Civilian firearm grade']==1,'b. mid','c. bottom'))
df['Military weapon grade'] = np.where(df['Weapon Free Funds: Military weapon grade']==0,'a. top',
                                       np.where(df['Weapon Free Funds: Military weapon grade']==1,'b. mid','c. bottom'))
df['Tobacco grade'] = np.where(df['Tobacco Free Funds: Tobacco grade']==0,'a. top',
                               np.where(df['Tobacco Free Funds: Tobacco grade']==1,'b. mid','c. bottom'))
df['Prison industrial complex grade'] = np.where(df['Prison Free Funds: Prison industrial complex grade']==0,'a. top',
                                                 np.where(df['Prison Free Funds: Prison industrial complex grade']==1,'b. mid','c. bottom'))

df['Grade'] = np.where((df['Fossil fuel grade']=='a. top') & (df['Deforestation grade']=='a. top') & (df['Gender equality grade']=='a. top') & 
                       (df['Civilian firearm grade']=='a. top') & (df['Military weapon grade']=='a. top') & (df['Tobacco grade']=='a. top') & 
                       (df['Prison industrial complex grade']=='a. top'), 'a. top',
                       np.where((df['Fossil fuel grade']=='c. bottom') | (df['Deforestation grade']=='c. bottom') | (df['Gender equality grade']=='c. bottom') | 
                                (df['Civilian firearm grade']=='c. bottom') | (df['Military weapon grade']=='c. bottom') | (df['Tobacco grade']=='c. bottom') | 
                                (df['Prison industrial complex grade']=='c. bottom'), 'c. bottom', 'b. mid'))

df1 = df.groupby('Grade').agg({'Fund profile: Fund name':'count'}).reset_index()
df1

,Grade,Fund profile: Fund name
0,a. top,4
1,b. mid,31
2,c. bottom,40


# 5. export dataset
* Fund name, Grade and Financial performance
* output dataset has 35996 rows and 12 columns

In [8]:
df = df[['Fund profile: Fund name', 'Fund profile: Category group', 'Fund profile: Fund net assets', 'Fossil fuel grade', 
         'Deforestation grade', 'Gender equality grade', 'Civilian firearm grade', 'Military weapon grade', 
         'Tobacco grade', 'Prison industrial complex grade', 'Grade', 'Financial performance']]
df.to_csv('../inputs/2_bins_financial_grade.csv', index=False)
df.shape

(75, 12)